In [1]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import hijri_converter
from hijri_converter import Hijri, Gregorian

In [2]:
from dateutil.relativedelta import relativedelta

In [3]:
def adjustable_range(start, stop=None, step=None):
    if stop is None:
        start, stop = 0, start

    if step is None: step = 1

    i = start
    while i < stop:
        change_bound = (yield i)
        if change_bound is None:
            i += step
        else:
            stop = change_bound

In [4]:
target_start_day_hijri = Hijri(1444,9,1) # Hijri date in the format 'yyyy-mm-dd'
target_end_day_hijri  = Hijri(1445,1,1) # Hijri date in the format 'yyyy-mm-dd'
is_different_year = (target_start_day_hijri.year!=target_end_day_hijri.year)

In [5]:
from hijri_converter import convert
def convert_date(date_string):
    return pd.to_datetime(date_string, errors = 'coerce')

# colNames = ['rid', 'Reservation Creation Date', 'Reservation Check-in Date', 'Reservation Check-out Date', 
#             'Cancellation Date', 'Allotment Type', 'Reservation Total Price', 'Rooms']
# colNames = ['rid', 'rcr_date', 'rci_date', 'rco_date', 'rtot_price', 'rnum_rooms', 'rcancel_date', 'rallot_type']
df = pd.read_excel("../datasets/Reservation List SA.xlsx", sheet_name="ReservationList", converters={'Reservation Check-out Date':convert_date  , 'Cancellation Date':convert_date , 'Reservation Check-in Date':convert_date ,'Reservation Creation Date':convert_date})



df.dropna(inplace=True, subset=['Reservation Creation Date', 'Reservation Check-in Date', 'Reservation Check-out Date'])
df['Hijri Check-out Date'] = df['Reservation Check-out Date'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())
df['Hijri Check-in Date'] = df['Reservation Check-in Date'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())


df.sort_values('Reservation Creation Date', inplace=True)

df['tmp'] = df['Reservation Creation Date'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())
df.insert(1, 'Reservation Creation Date Hijri', df.pop('tmp'))

df.reset_index(inplace=True, drop=True)
df

,Reservation ID,Reservation Creation Date Hijri,Reservation Creation Date,Reservation Check-in Date,Reservation Check-out Date,Reservation Total Price,Rooms,Cancellation Date,Allotment Type,Hijri Check-out Date,Hijri Check-in Date
0,2,1442-12-17,2021-07-27,2021-07-31,2021-08-03,2545.2000,1,NaT,1,1442-12-24,1442-12-21
1,1,1442-12-17,2021-07-27,2021-08-22,2021-08-24,938.7000,2,NaT,1,1443-01-16,1443-01-14
2,3,1442-12-22,2021-08-01,2021-08-05,2021-08-07,1202.9000,2,2021-08-01 13:52:42,1,1442-12-28,1442-12-26
3,5,1442-12-22,2021-08-01,2021-08-12,2021-08-17,2594.6000,2,NaT,1,1443-01-09,1443-01-04
4,4,1442-12-22,2021-08-01,2021-08-12,2021-08-14,917.7000,3,NaT,1,1443-01-06,1443-01-04
...,...,...,...,...,...,...,...,...,...,...,...
83191,83086,1444-12-27,2023-07-15,2023-11-16,2023-11-20,3900.1100,4,NaT,1,1445-05-06,1445-05-02
83192,83037,1444-12-27,2023-07-15,2023-07-11,2023-07-13,799.9998,1,NaT,1,1444-12-25,1444-12-23
83193,83119,1444-12-27,2023-07-15,2023-07-19,2023-07-24,3479.9920,1,NaT,1,1445-01-06,1445-01-01
83194,83075,1444-12-27,2023-07-15,2023-11-02,2023-11-06,2280.0360,4,NaT,1,1445-04-22,1445-04-18


In [6]:
df.isna().sum()

Reservation ID                         0
Reservation Creation Date Hijri        0
Reservation Creation Date              0
Reservation Check-in Date              0
Reservation Check-out Date             0
Reservation Total Price                0
Rooms                                  0
Cancellation Date                  57998
Allotment Type                         0
Hijri Check-out Date                   0
Hijri Check-in Date                    0
dtype: int64

In [7]:
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg

def greg_to_hijri(greg_date):
    hijri_date = Gregorian(greg_date.year, greg_date.month, greg_date.day)
    hijri_date = hijri_date.to_hijri()
    return hijri_date

def increment_hijri(hijri_date, dateOffset=1):
    dt_greg = hijri_to_greg(hijri_date)
    dt_greg += pd.DateOffset(days=dateOffset)
    incremented_hijri_date = Gregorian(dt_greg.year, dt_greg.month, dt_greg.day).to_hijri()
    return incremented_hijri_date

In [8]:
def expand_checkinout_range_to_target_dates(df):

    # Create an empty list to store the expanded data of the reservations's checkin/checkout range
    rows = []

    # Iterate over each row in the original DataFrame
    for index, row in df.iterrows():
        # Extract the check-in and check-out  Hijri dates 
        check_in_date_hijri = row['Hijri Check-in Date']
        check_out_date_hijri = row['Hijri Check-out Date']
        target_start_day_hijri_temp = Hijri(check_in_date_hijri.year, target_start_day_hijri.month, target_start_day_hijri.day) 
        target_end_day_hijri_temp = Hijri(check_out_date_hijri.year + is_different_year, target_end_day_hijri.month, target_end_day_hijri.day)

        # Iterate over each day within the range
        current_day = check_in_date_hijri
        while current_day <= check_out_date_hijri:
            # Check if the current day falls within the check-in and check-out dates
            if target_start_day_hijri_temp <= current_day <= target_end_day_hijri_temp:
                # Append the row with the current day to the list
                rows.append(list(row)+[current_day])
            current_day = increment_hijri(current_day)

    # Create the expanded DataFrame directly from the list of rows
    expanded_df = pd.DataFrame(rows, columns=list(df.columns)+['Target Date'])

    
    return expanded_df

In [9]:
expanded_df = expand_checkinout_range_to_target_dates(df)

In [10]:
def drop_reservations_for_checkins_in_different_years(df):
    df = df.copy()
    tot_rows = len(df)
    df['hijri_rcr_year'] = df['Reservation Creation Date Hijri'].apply(lambda x:x.year)
    df['hijri_target_year'] = df['Target Date'].apply(lambda x:x.year)
    df = df[df['hijri_rcr_year'] == df['hijri_target_year']]
    rem_rows = tot_rows - len(df)
    print(f'dropped {rem_rows} reservations which have check in/out in a different year')
    return df

In [11]:
expanded_df = drop_reservations_for_checkins_in_different_years(expanded_df)

dropped 1858 reservations which have check in/out in a different year


In [12]:
float_columns = [col for col in expanded_df.columns if 'date' not in col.lower()]
int_columns = [col for col in expanded_df.columns if 'year' in col.lower()]
expanded_df[float_columns] = expanded_df[float_columns].apply(pd.to_numeric, errors='coerce').astype('float')
expanded_df[int_columns] = expanded_df[int_columns].apply(pd.to_numeric, errors='coerce').astype('int')
print(expanded_df.dtypes)

Reservation ID                            float64
Reservation Creation Date Hijri            object
Reservation Creation Date          datetime64[ns]
Reservation Check-in Date          datetime64[ns]
Reservation Check-out Date         datetime64[ns]
Reservation Total Price                   float64
Rooms                                     float64
Cancellation Date                  datetime64[ns]
Allotment Type                            float64
Hijri Check-out Date                       object
Hijri Check-in Date                        object
Target Date                                object
hijri_rcr_year                              int32
hijri_target_year                           int32
dtype: object


In [13]:
def get_cr_dates_per_year(df, are_dates_hijri=False):
    """
    Retrieve a list of DataFrame arrays containing reservation creation dates grouped by year.

    Parameters:
        df (pandas.DataFrame): The initial DataFrame.
        are_dates_hijri (bool): Flag indicating whether the dates are in Hijri format. 
                                Default is True.

    Returns:
        list: A list of DataFrame arrays, where each array contains reservation creation dates for a specific year.

    Example:
        >>> df = pd.DataFrame({
        ...     'Reservation Creation Date (MM-DD-YYYY)': ['01-01-2022', '01-02-2022', '01-01-2023', '01-02-2023'],
        ...     'Reservation Creation Date Hijri': ['1444-01-01', '1444-01-02', '1444-01-01', '1444-01-02']
        ... })
        >>> cr_dates_per_year_gregorian = get_cr_dates_per_year(df)
        >>> print(cr_dates_per_year_gregorian)
        [['01-01-2022', '01-02-2022'], ['01-01-2023', '01-02-2023']]
        >>> cr_dates_per_year_hijri = get_cr_dates_per_year(df, True)
        >>> print(cr_dates_per_year_hijri)
        [['1444-01-01', '1444-01-02']]
    """

    df = df.copy()

    if are_dates_hijri:
        df['Hijri Year'] = df['Reservation Creation Date Hijri'].apply(lambda x: x.year)
        # Group the DataFrame by year
        grouped = df.groupby(df['Hijri Year'])

        # Create separate DataFrame arrays for each year
        cr_dates_per_year = [group['Reservation Creation Date Hijri'].tolist() for _, group in grouped]

    else:
        # Group the DataFrame by year
        grouped = df.groupby(df['Reservation Creation Date'].dt.year)

        # Create separate DataFrame arrays for each year
        cr_dates_per_year = [group['Reservation Creation Date'].tolist() for _, group in grouped]
    cr_dates_per_year = [list(OrderedDict.fromkeys(dates_in_a_year)) for dates_in_a_year in cr_dates_per_year]
    return cr_dates_per_year

In [14]:
def get_hijri_dates_between_range(start_date, end_date):
    """
    Generate a list of Hijri dates between the given start and end dates (inclusive).

    Parameters:
        start_date (Hijri): The starting Hijri date of the range.
        end_date (Hijri): The ending Hijri date of the range.

    Returns:
        list: A list of Hijri dates between the start and end dates (inclusive).

    Example:
        >>> from hijri_converter import Hijri
        >>> start_date = Hijri(1444, 1, 1)
        >>> end_date = Hijri(1444, 1, 5)
        >>> hijri_dates_range = get_hijri_dates_between_range(start_date, end_date)
        >>> print(hijri_dates_range)
        [Hijri(1444, 1, 1), Hijri(1444, 1, 2), Hijri(1444, 1, 3), Hijri(1444, 1, 4), Hijri(1444, 1, 5)]
    """
    dates_range = []
    current_day = start_date
    while current_day <= end_date:
        dates_range.append(current_day)
        current_day = increment_hijri(current_day)
    return dates_range


In [15]:
def create_reservation_dataframe(year, is_year_hijri=False, end_date_at_year_beginning=False):
    """
    Create a dataframe with a column 'Reservation Creation Date' containing all the days in a specified year.

    Parameters:
        year (int): The year for which to create the dataframe.
        is_year_hijri (bool, optional): Flag indicating whether the year is in Hijri format. Default is False.
        end_date_at_year_beginning (bool, optional): Flag indicating whether to set the end date at the beginning of the next year.
                                                    Applicable only if is_year_hijri is True. Default is False.

    Returns:
        pandas.DataFrame: A dataframe with a single column 'Reservation Creation Date' containing all the days in the specified year.

    Example:
        >>> # Example with Gregorian dates
        >>> df_gregorian = create_reservation_dataframe(2023)
        >>> print(df_gregorian)
           Reservation Creation Date
        0                2023-01-01
        1                2023-01-02
        2                2023-01-03
        ...              ...
        364              2023-12-31

        >>> # Example with Hijri dates
        >>> from hijri_converter import Hijri
        >>> df_hijri = create_reservation_dataframe(1444, is_year_hijri=True)
        >>> print(df_hijri)
           Reservation Creation Date
        0                1444-01-01
        1                1444-01-02
        2                1444-01-03
        ...              ...
        354              1444-12-24
        355              1444-12-25
        356              1444-12-26
        357              1444-12-27
        358              1444-12-28
        359              1444-12-29
        360              1444-12-30
        361              1444-12-31
    """
    if is_year_hijri:
        start_date = Hijri(year, 1, 1)
        if end_date_at_year_beginning:
            end_date = Hijri(year+1, 1, 1)
        else:
            end_date = Hijri(year, 12, Hijri(year, 12, 1).month_length())
        dates = get_hijri_dates_between_range(start_date, end_date)
    else:
        start_date = pd.to_datetime(f'{year}-01-01')
        end_date = pd.to_datetime(f'{year}-12-31')
        dates = pd.date_range(start=start_date, end=end_date, freq='D')

    df = pd.DataFrame({'Reservation Creation Date': dates})

    return df

In [16]:
def get_target_dates_history_dfs_and_years(expanded_df, add_start_of_new_year_as_a_row=True):
    """
    Create target dates history DataFrames and their corresponding years.

    This function processes the filtered DataFrame containing reservation data with Hijri dates.
    It generates target dates history DataFrames for each year in the filtered data and stores them
    along with their corresponding years in separate lists.

    Parameters:
        expanded_df (pandas.DataFrame): The filtered DataFrame containing reservation data with Hijri dates.
        add_start_of_new_year_as_a_row (bool, optional): Flag indicating whether to add a row for the start of the new year
                                                         in the target dates history DataFrames. Default is True.

    Returns:
        tuple: A tuple containing two lists - target_dates_history_dfs and target_dates_history_years.
                target_dates_history_dfs (list): A list of DataFrames containing target dates history
                                                for each year in the filtered data.
                target_dates_history_years (list): A list of integers representing the years for which
                                                    the target dates history DataFrames are generated.

    Example:
        >>> # Sample DataFrame with Hijri dates
        >>> expanded_df = pd.DataFrame({
        ...     'Reservation Creation Date Hijri': ['1444-01-01', '1444-01-01', '1444-01-02', '1445-02-17'],
        ...     'Target Date': ['1444-01-03', '1444-01-03', '1445-01-04', '1445-01-05'],
        ...     'Rooms': [3, 2, 7, 8]
        ... })
        >>> target_start_day_hijri = Hijri(1444, 1, 1)
        >>> target_end_day_hijri = Hijri(1444, 1, 5)
        >>> is_different_year = 0
        >>> target_dates_history_dfs, target_dates_history_years = get_target_dates_history_dfs_and_years(
        ...     expanded_df, add_start_of_new_year_as_a_row=True
        ... )
        >>> print(target_dates_history_years)
        [1444]
        >>> print(target_dates_history_dfs[0])
            Reservation Creation Date  1444-01-01  1444-01-02  1444-01-03  1444-01-04  1444-01-05
        0                  1444-01-01         NaN         NaN         5.0         NaN         NaN
        1                  1444-01-02         NaN         NaN         NaN         7.0         NaN
        ...
        365                1445-01-01         NaN         NaN         NaN         NaN         NaN
        >>> Note: the last row here will be 1444-12-30 if the the add_start_of_new_year_as_a_row flag is False.
        """
    global target_start_day_hijri, target_end_day_hijri, is_different_year
    cr_dates_per_year = get_cr_dates_per_year(expanded_df, are_dates_hijri=True)
    target_dates_history_years = []
    target_dates_history_dfs = []
    for annually_creation_dates_hijri in cr_dates_per_year:
        target_start_day_hijri_temp = Hijri(annually_creation_dates_hijri[0].year, 
                                            target_start_day_hijri.month, 
                                            target_start_day_hijri.day)
        target_end_day_hijri_temp = Hijri(annually_creation_dates_hijri[0].year + is_different_year, 
                                            target_end_day_hijri.month, 
                                            target_end_day_hijri.day)
        
        target_dates_range = get_hijri_dates_between_range(target_start_day_hijri_temp, target_end_day_hijri_temp)

        # Creating empty df
        annually_target_dates_history_df = create_reservation_dataframe(annually_creation_dates_hijri[0].year, 
                                                                        is_year_hijri=True, end_date_at_year_beginning=add_start_of_new_year_as_a_row)
        # Creates columns where the column name is a Hijri() object (displayed as Y-m-d by __str()__ method in Hijri class)
        for column in target_dates_range:
            annually_target_dates_history_df[column] = np.nan
        
        for creation_date_hijri in annually_creation_dates_hijri:
            target_creation_dates_mask = (annually_target_dates_history_df['Reservation Creation Date'] == creation_date_hijri)
            filtered_creation_dates_mask = (expanded_df['Reservation Creation Date Hijri'] == creation_date_hijri)
            filtered_target_dates_start_range_mask = (expanded_df['Target Date'] >= Hijri(creation_date_hijri.year, target_start_day_hijri.month, target_start_day_hijri.day))
            filtered_target_dates_end_range_mask = (expanded_df['Target Date'] < Hijri(creation_date_hijri.year+1, target_end_day_hijri.month, target_end_day_hijri.day))
            
            tmp_filtered_df = expanded_df[filtered_creation_dates_mask & filtered_target_dates_start_range_mask & filtered_target_dates_end_range_mask]
            filtered_target_dates = list(tmp_filtered_df['Target Date'])
            filtered_rooms = list(tmp_filtered_df['Rooms'])
            
            target_date_to_rooms = {}
            for i, target_date in enumerate(filtered_target_dates):
                if target_date not in target_date_to_rooms.keys():
                    target_date_to_rooms[target_date] = 0
                target_date_to_rooms[target_date] += filtered_rooms[i]    
                                  
            annually_target_dates_history_df.loc[target_creation_dates_mask, # to select row in ann_td_h_df where creation date is the current one in the loop
                                                    list(target_date_to_rooms.keys()) # to update the column names of said row
                                                ] \
                                                = list(target_date_to_rooms.values()) # updates it to Rooms

        target_dates_history_dfs.append(annually_target_dates_history_df)
        target_dates_history_years.append(annually_creation_dates_hijri[0].year)

    return target_dates_history_dfs, target_dates_history_years

In [18]:
target_dates_history_dfs, target_dates_history_years = get_target_dates_history_dfs_and_years(expanded_df, add_start_of_new_year_as_a_row=True)

C:\Users\ashra\AppData\Local\Temp\ipykernel_66176\1758329747.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annually_target_dates_history_df[column] = np.nan
C:\Users\ashra\AppData\Local\Temp\ipykernel_66176\1758329747.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annually_target_dates_history_df[column] = np.nan
C:\Users\ashra\AppData\Local\Temp\ipykernel_66176\1758329747.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [19]:
target_dates_history_dfs[0]

,Reservation Creation Date,1442-09-01,1442-09-02,1442-09-03,1442-09-04,1442-09-05,1442-09-06,1442-09-07,1442-09-08,1442-09-09,...,1442-12-21,1442-12-22,1442-12-23,1442-12-24,1442-12-25,1442-12-26,1442-12-27,1442-12-28,1442-12-29,1443-01-01
0,1442-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1442-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1442-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1442-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1442-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,1442-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,1442-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,1442-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,1442-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
def calculate_target_dates_accumulation(target_dates_history_dfs):
    """
    Calculate the target dates' accumulation in the target dates history DataFrames.

    This function takes a list of DataFrames, each representing the target dates history for a specific year,
    and calculates the cumulative target dates accumulation in each DataFrame.

    Parameters:
        target_dates_history_dfs (list): A list of DataFrames containing target dates history
                                         for each year.

    Returns:
        list: A list of DataFrames with the cumulative target dates accumulation for each year.

    Example:
        >>> # Assuming target_dates_history_dfs is the result of get_target_dates_history_dfs_and_years()
        >>> target_dates_accum_history_dfs = target_dates_history_dfs.copy()
        >>> target_dates_accum_history_dfs = calculate_target_dates_accumulation(target_dates_accum_history_dfs)
        >>> for tdacc_df in target_dates_accum_history_dfs:
        ...     print(tdacc_df)
                    Reservation Creation Date  1444-01-01  1444-01-02  1444-01-03  1444-01-04  1444-01-05
            0                  1444-01-01         0.0         0.0         5.0         0.0         0.0
            1                  1444-01-02         0.0         0.0         NaN         7.0         0.0
            ...
            365                1445-01-01         NaN         NaN         NaN         NaN         NaN
    """
    target_dates_accum_history_dfs = []
    for df in target_dates_history_dfs:
        target_dates_accum_history_dfs.append(df.copy())

    for tdacc_df in target_dates_accum_history_dfs:
        # cumulatively add each row to the row below it
        tdacc_df.fillna(0, inplace=True)
        tdacc_df[tdacc_df.columns[1:]] = tdacc_df[tdacc_df.columns[1:]].cumsum()

        # when reaching the row where "Reservation Creation Date" is equal to the Column date, 
        # fill that column's remaining rows (after that row) with 0s 
        for i, column_hijri in enumerate(tdacc_df.columns):
            if not isinstance(column_hijri, Hijri):
                continue 
            matching_dates_row_idx = tdacc_df.loc[tdacc_df['Reservation Creation Date']==column_hijri].index[0]
            if matching_dates_row_idx == len(tdacc_df)-1:
                continue
            tdacc_df.iloc[matching_dates_row_idx+1:, i] = np.nan

    return target_dates_accum_history_dfs

In [21]:
target_dates_accum_history_dfs = calculate_target_dates_accumulation(target_dates_history_dfs)

In [22]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

In [23]:
target_dates_history_dfs[0]

,Reservation Creation Date,1442-09-01,1442-09-02,1442-09-03,1442-09-04,1442-09-05,1442-09-06,1442-09-07,1442-09-08,1442-09-09,...,1442-12-21,1442-12-22,1442-12-23,1442-12-24,1442-12-25,1442-12-26,1442-12-27,1442-12-28,1442-12-29,1443-01-01
0,1442-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1442-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1442-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1442-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1442-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,1442-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,1442-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,1442-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,1442-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
def process_weekly_target_dates_accumulation(target_dates_accum_history_dfs):
    """
    Process weekly target dates accumulation DataFrames.

    This function takes a list of DataFrames, each representing the target dates accumulation for a specific year,
    and processes them to create weekly target dates accumulation DataFrames. Each weekly DataFrame includes aggregated
    data for a week's range of target dates, with trailing zero values converted to NaN.

    Parameters:
        target_dates_accum_history_dfs (list): A list of DataFrames containing cumulative target dates accumulation
                                               for each year.

    Returns:
        list: A list of DataFrames with weekly target dates accumulation for each year.

        >>> # Sample output of weekly_target_dates_accum_dfs for one year
        >>> print(weekly_target_dates_accum_dfs[0])
            Reservation Creation Date  W1 - Ramadhan      W2 - Ramadhan  ...  W3 - Dhu al-Hijjah  W4 - Dhu al-Hijjah  W1 - Dhu al-Hijjah/Muharram
        0                  1441-01-01          0.0                  0.0  ...           0.0           0.0                     0.0
        1                  1441-01-02          0.0                  0.0  ...           0.0           0.0                     0.0
        2                  1441-01-03          5.0                  0.0  ...           0.0           0.0                     0.0
        ...                ...               ...                    ...  ...           ...           ...                     ...
        353                1441-12-28          NaN                  NaN  ...           NaN           NaN                     NaN
        354                1441-12-29          NaN                  NaN  ...           NaN           NaN                     NaN
        355                1442-01-01          NaN                  NaN  ...           NaN           NaN                     NaN

    Note: The DataFrames in weekly_target_dates_accum_dfs will contain aggregated values for each week's target date range,
    with trailing zero values converted to NaN to avoid displaying inaccurate data.
    """
    weekly_target_dates_accum_dfs = []
    for tdacc_df in target_dates_accum_history_dfs:
        new_columns = ['Reservation Creation Date']
        new_agg_values = [tdacc_df.iloc[:, 0].to_list()]
        df_columns = tdacc_df.columns
        j = 0
        start_date_idx = 1
        while start_date_idx < len(df_columns):
            #incrment j by one to detect last week 
            j += 1 
            # get column name of this week's range
            end_date_idx = min(start_date_idx+6, len(df_columns)-1)
            week_num = j
            if week_num % 4 == 0:
                week_num = 4
            else:
                week_num %= 4
            column_name = f'W{week_num} - ' + df_columns[start_date_idx].month_name()
            # if df_columns[start_date_idx].month != df_columns[end_date_idx].month:
            #     column_name += '/' + df_columns[end_date_idx].month_name()
            new_columns.append(column_name)
            
            # if this is the last week in the month add all the remaning days in current month to last week
            if j % 4 == 0:
                mlen = tdacc_df.columns[start_date_idx].month_length()
                tlen = mlen - 21
                # get sum of Rooms of this week's range as a single list
                new_agg_values.append(tdacc_df.iloc[:, start_date_idx:start_date_idx+tlen].sum(axis=1, skipna=True).to_list())
                start_date_idx += tlen - 7 
            else:
                new_agg_values.append(tdacc_df.iloc[:, start_date_idx:start_date_idx+7].sum(axis=1, skipna=True).to_list())
            start_date_idx += 7
            
        weekly_tdacc_df = pd.DataFrame(np.array(new_agg_values).T, columns=new_columns)

        # convert all trailing zero values (per column) to np.NaN values
        for column in weekly_tdacc_df.columns:
            if 'date' in column.lower():
                continue
            mask = weekly_tdacc_df[column] != 0
            last_nonzero_index = mask[::-1].idxmax()
            if last_nonzero_index < len(weekly_tdacc_df)-1:
                weekly_tdacc_df.loc[last_nonzero_index + 1:, column] = np.nan

        weekly_target_dates_accum_dfs.append(weekly_tdacc_df)
    
    for df in weekly_target_dates_accum_dfs:
        df.drop(df.columns[-1], axis=1, inplace=True)

    return weekly_target_dates_accum_dfs

In [25]:
weekly_target_dates_accum_dfs = process_weekly_target_dates_accumulation(target_dates_accum_history_dfs)

In [26]:
weekly_target_dates_accum_dfs[0]

,Reservation Creation Date,W1 - Ramadhan,W2 - Ramadhan,W3 - Ramadhan,W4 - Ramadhan,W1 - Shawwal,W2 - Shawwal,W3 - Shawwal,W4 - Shawwal,W1 - Dhu al-Qi’dah,W2 - Dhu al-Qi’dah,W3 - Dhu al-Qi’dah,W4 - Dhu al-Qi’dah,W1 - Dhu al-Hijjah,W2 - Dhu al-Hijjah,W3 - Dhu al-Hijjah,W4 - Dhu al-Hijjah
0,1442-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1442-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1442-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1442-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1442-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,1442-12-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,9.0
351,1442-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,7.0
352,1442-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0
353,1442-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0


In [27]:
# visualized example in markdowen below |
#                                       v
def handle_accumulated_rooms(weekly_target_dates_accum_dfs):
    """
    This function deals with a corner case where the accumulated rooms for each week
    get decreased in the rows corresponding to the target week. It updates the accumulated
    room values to address this discrepancy.

    Parameters:
        weekly_target_dates_accum_dfs (list of DataFrames): A list of pandas DataFrames
        containing weekly data for accumulated rooms with columns representing dates.

    Explanation:
        The function iterates through each DataFrame in the provided list, representing
        the accumulated rooms for different weeks. It then processes each column (date) in
        the DataFrame, excluding columns containing 'date' in their name.

        For each column, it finds the starting index of the target week by locating the last
        non-null value in that column (the last recorded value for the week) and subtracting 6
        (representing 6 days in a week) to get the index of the first day of the target week.

        Next, it iterates over 7 days (a week) starting from the target week's start index and
        updates the accumulated room values for each day. The update is based on the previous day's
        value and the difference between the previous day's value and the current day's value.

        The purpose of this process is to handle the case where there might be a discrepancy in
        the accumulated room values for the target week, resulting in a decrease. This discrepancy
        could occur due to adjustments or corrections made to the data.

    Note:
        - This code assumes that the DataFrames in the input list have columns representing dates.
        - The provided 'weekly_target_dates_accum_dfs' list is modified in-place, and no new list
          is returned.

    Example usage:
        weekly_data = [...]  # List of DataFrames containing weekly accumulated room data
        handle_accumulated_rooms(weekly_data)
    """
    for df in weekly_target_dates_accum_dfs:
        for col in df.columns:
            if 'date' in col.lower():
                continue
            current_target_week_start_idx = df.loc[:, col].dropna().index[-1] - 6
            for i in range(current_target_week_start_idx+1, current_target_week_start_idx+7):
                df.loc[i, col] += df.loc[i-1, col] - df.loc[i, col]


handle_accumulated_rooms() explanation by example:

Before:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; After:


<img src="../project_media/before_acc.png" alt="Alt Text" width="300" >  
<img src="../project_media/after_acc_fixed.png" alt="Alt Text" width="300">




In [28]:
handle_accumulated_rooms(weekly_target_dates_accum_dfs)

In [29]:
weekly_target_dates_accum_dfs[0]

,Reservation Creation Date,W1 - Ramadhan,W2 - Ramadhan,W3 - Ramadhan,W4 - Ramadhan,W1 - Shawwal,W2 - Shawwal,W3 - Shawwal,W4 - Shawwal,W1 - Dhu al-Qi’dah,W2 - Dhu al-Qi’dah,W3 - Dhu al-Qi’dah,W4 - Dhu al-Qi’dah,W1 - Dhu al-Hijjah,W2 - Dhu al-Hijjah,W3 - Dhu al-Hijjah,W4 - Dhu al-Hijjah
0,1442-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1442-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1442-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1442-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1442-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,1442-12-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.0
351,1442-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.0
352,1442-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.0
353,1442-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.0


In [30]:
len(weekly_target_dates_accum_dfs)

3

For months

In [35]:
def get_monthly_target_dates_accumulation(weekly_target_dates_accum_dfs): 
    """
    Process weekly target dates accumulation DataFrames (dfs) into monthly dfs.

    This function takes a list of DataFrames, each representing the target dates accumulation for a specific year,
    and processes them to create weekly target dates accumulation DataFrames. Each weekly DataFrame includes aggregated
    data for a week's range of target dates, with trailing zero values converted to NaN.

    Parameters:
        target_dates_accum_history_dfs (list): A list of DataFrames containing cumulative target dates accumulation
                                               for each year.

    Returns:
        list: A list of DataFrames with weekly target dates accumulation for each year.

        >>> # Sample output of weekly_target_dates_accum_dfs for one year
        >>> print(weekly_target_dates_accum_dfs[0])
            Reservation Creation Date  W1 - Ramadhan      W2 - Ramadhan  ...  W3 - Dhu al-Hijjah  W4 - Dhu al-Hijjah  W1 - Dhu al-Hijjah/Muharram
        0                  1441-01-01          0.0                  0.0  ...           0.0           0.0                     0.0
        1                  1441-01-02          0.0                  0.0  ...           0.0           0.0                     0.0
        2                  1441-01-03          5.0                  0.0  ...           0.0           0.0                     0.0
        ...                ...               ...                    ...  ...           ...           ...                     ...
        353                1441-12-28          NaN                  NaN  ...           NaN           NaN                     NaN
        354                1441-12-29          NaN                  NaN  ...           NaN           NaN                     NaN
        355                1442-01-01          NaN                  NaN  ...           NaN           NaN                     NaN

    Note: The DataFrames in weekly_target_dates_accum_dfs will contain aggregated values for each week's target date range,
    with trailing zero values converted to NaN to avoid displaying inaccurate data.
    """
    monthly_target_dates_accum_dfs = []
    for wtdacc_df in weekly_target_dates_accum_dfs:
        new_columns = ['Reservation Creation Date']
        new_agg_values = [wtdacc_df.iloc[:, 0].to_list()]
        df_columns = wtdacc_df.columns
        for start_date_idx in range(1, len(df_columns), 4):
            # get column name of this week's range
            end_date_idx = min(start_date_idx+4, len(df_columns)-1)
            column_name = f'M{(start_date_idx//4) + 1} - ' + df_columns[start_date_idx].split(' - ')[-1]
            new_columns.append(column_name)

            # get sum of Rooms of this week's range as a single list
            new_agg_values.append(wtdacc_df.iloc[:, start_date_idx:start_date_idx+4].sum(axis=1, skipna=True).to_list())
            
        monthly_tdacc_df = pd.DataFrame(np.array(new_agg_values).T, columns=new_columns)

        # convert all trailing zero values (per column) to np.NaN values
        for column in monthly_tdacc_df.columns:
            if 'date' in column.lower():
                continue
            mask = monthly_tdacc_df[column] != 0
            last_nonzero_index = mask[::-1].idxmax()
            if last_nonzero_index < len(monthly_tdacc_df)-1:
                monthly_tdacc_df.loc[last_nonzero_index + 1:, column] = np.nan

        monthly_target_dates_accum_dfs.append(monthly_tdacc_df)
        
    return monthly_target_dates_accum_dfs

In [36]:
monthly_target_dates_accum_dfs = get_monthly_target_dates_accumulation(weekly_target_dates_accum_dfs)

In [37]:
monthly_target_dates_accum_dfs[0]

,Reservation Creation Date,M1 - Ramadhan,M2 - Shawwal,M3 - Dhu al-Qi’dah,M4 - Dhu al-Hijjah
0,1442-01-01,0.0,0.0,0.0,0.0
1,1442-01-02,0.0,0.0,0.0,0.0
2,1442-01-03,0.0,0.0,0.0,0.0
3,1442-01-04,0.0,0.0,0.0,0.0
4,1442-01-05,0.0,0.0,0.0,0.0
...,...,...,...,...,...
350,1442-12-26,0.0,0.0,0.0,8.0
351,1442-12-27,0.0,0.0,0.0,8.0
352,1442-12-28,0.0,0.0,0.0,8.0
353,1442-12-29,0.0,0.0,0.0,8.0


In [38]:
def handle_accumulated_rooms_monthly(monthly_target_dates_accum_dfs):
    """
    This function deals with a corner case where the accumulated rooms for each week
    get decreased in the rows corresponding to the target week. It updates the accumulated
    room values to address this discrepancy.

    Parameters:
        weekly_target_dates_accum_dfs (list of DataFrames): A list of pandas DataFrames
        containing weekly data for accumulated rooms with columns representing dates.

    Explanation:
        The function iterates through each DataFrame in the provided list, representing
        the accumulated rooms for different weeks. It then processes each column (date) in
        the DataFrame, excluding columns containing 'date' in their name.

        For each column, it finds the starting index of the target week by locating the last
        non-null value in that column (the last recorded value for the week) and subtracting 6
        (representing 6 days in a week) to get the index of the first day of the target week.

        Next, it iterates over 7 days (a week) starting from the target week's start index and
        updates the accumulated room values for each day. The update is based on the previous day's
        value and the difference between the previous day's value and the current day's value.

        The purpose of this process is to handle the case where there might be a discrepancy in
        the accumulated room values for the target week, resulting in a decrease. This discrepancy
        could occur due to adjustments or corrections made to the data.

    Note:
        - This code assumes that the DataFrames in the input list have columns representing dates.
        - The provided 'weekly_target_dates_accum_dfs' list is modified in-place, and no new list
          is returned.

    Example usage:
        weekly_data = [...]  # List of DataFrames containing weekly accumulated room data
        handle_accumulated_rooms(weekly_data)
    """
    for df in monthly_target_dates_accum_dfs:
        for col in df.columns:
            if 'date' in col.lower():
                continue
            current_target_month_start_idx = df.loc[:, col].dropna().index[-1] - 29
            for i in range(current_target_month_start_idx+1, current_target_month_start_idx+30):
                df.loc[i, col] += df.loc[i-1, col] - df.loc[i, col]


In [39]:
handle_accumulated_rooms_monthly(monthly_target_dates_accum_dfs)

In [40]:
monthly_target_dates_accum_dfs[0]

,Reservation Creation Date,M1 - Ramadhan,M2 - Shawwal,M3 - Dhu al-Qi’dah,M4 - Dhu al-Hijjah
0,1442-01-01,0.0,0.0,0.0,0.0
1,1442-01-02,0.0,0.0,0.0,0.0
2,1442-01-03,0.0,0.0,0.0,0.0
3,1442-01-04,0.0,0.0,0.0,0.0
4,1442-01-05,0.0,0.0,0.0,0.0
...,...,...,...,...,...
350,1442-12-26,0.0,0.0,0.0,0.0
351,1442-12-27,0.0,0.0,0.0,0.0
352,1442-12-28,0.0,0.0,0.0,0.0
353,1442-12-29,0.0,0.0,0.0,0.0


pipline for normal target dates (not accum)

In [42]:
weekly_target_dates_dfs = process_weekly_target_dates_accumulation(target_dates_history_dfs)
handle_accumulated_rooms(weekly_target_dates_dfs)
monthly_target_dates_dfs = get_monthly_target_dates_accumulation(weekly_target_dates_dfs)

In [43]:
monthly_target_dates_dfs[0]

,Reservation Creation Date,M1 - Ramadhan,M2 - Shawwal,M3 - Dhu al-Qi’dah,M4 - Dhu al-Hijjah
0,1442-01-01,0.0,0.0,0.0,0.0
1,1442-01-02,0.0,0.0,0.0,0.0
2,1442-01-03,0.0,0.0,0.0,0.0
3,1442-01-04,0.0,0.0,0.0,0.0
4,1442-01-05,0.0,0.0,0.0,0.0
...,...,...,...,...,...
350,1442-12-26,0.0,0.0,0.0,NaN
351,1442-12-27,0.0,0.0,0.0,NaN
352,1442-12-28,0.0,0.0,0.0,NaN
353,1442-12-29,0.0,0.0,0.0,NaN


# Saving cleaned dataframes to Excel files

In [45]:
def save_dfs_to_excel(dfs, file_prefix):
    global target_dates_history_years
    for index, df in enumerate(dfs, start=0):
        file_name = f"{file_prefix}_{target_dates_history_years[index]}.xlsx"
        df.to_excel(file_name, index=False)

In [46]:
save_dfs_to_excel(target_dates_history_dfs, 'target_dates_history_df')
save_dfs_to_excel(weekly_target_dates_dfs, 'weekly_target_dates_df')
save_dfs_to_excel(weekly_target_dates_accum_dfs, 'weekly_target_dates_accum_df')
save_dfs_to_excel(monthly_target_dates_dfs, 'monthly_target_dates_df')
save_dfs_to_excel(monthly_target_dates_accum_dfs, 'monthly_target_dates_accum_df')